# ECDSA - Elliptic Curve Digital Signature Algorithm

On this notebook you can find a brief introduction to ECDSA. How it works, and why it works!  

If you want to know more about Digital Signatures, [check this out](https://cryptobook.nakov.com/digital-signatures/) !

### What's a Digital Signature Algorithm (DSA)?
It is a cryptographic algorithm used to generate digital signatures, authenticate the sender of a digital message, and prevent message tampering.  
DSA works by having two keys: a **private key** owned by the **sender** and a **public key** held by the **receiver**.

### Key Generation
The ECDSA key-pair consists of:
- private key: integer
- public key: pubKey = privKey * G, where G is an elliptic curve point

### ECDSA Sign
The ECDSA signing algorithm takes as input a message ***msg*** + a private key ***privKey*** and produces as output a **signature**, which consists of pair of integers **{r, s}**.  

The ECDSA is based on the ElGamal Signature Scheme and works as follows:  
**1.** Calculate the message hash, using a cryptographic hash function (we will use SHA-256): **h = hash(msg)**  
**2.** Generate securely a random number **k** in the range [1..n-1], where n is the order of the curve  
**3.** Calculate the random point **R = k * G** and take its x-coordinate: **r = R.x**  
**4.** Calculate the signature proof: $$ s = k^{-1} * (h + r * privKey) \mod n$$  
**5.** Return the signature **{r,s}**

### ECDSA Verify Signature
The algorithm to verify a ECDSA signature takes as **input** the signed message **msg** + the signature **{r, s}** produced from the **signing** algorithm + the public key **pubKey**, corresponding to the **signer's private key**.  

The ECDSA signature verify algorithm works as follows:  

**1.** Calculate the message hash, with the same cryptographic hash function used during the signing: **h = hash(msg)**  
**2.** Calculate the modular inverse of the signature proof: $$ s1 = s^{-1}(\mod n) $$  
**3.** Recover the random point used during the signing: $$ R' = (h*s1)*G + (r*s1) * pubKey$$  
**4.** Take from R' its x-coordinate: $$ r' = R'.x $$  
**5.** Calculate the signature validation result by comparing wether **r' == r**

### But... Does it really work?
Well lets look at the Maths!

The equation behind the recovering of the point **R'**, calculated during the **signature verification**, can be transformed by replacing the **pubKey** with **privKey * G** as follows:

$$ R' = (h*s1)*G + (r*s1) * pubKey $$
$$ \equiv $$
$$R' = (h*s1)*G + (r * s1) * privKey * G$$
$$ \equiv $$
$$R' = (h + r* privKey) * s1 * G$$  

If we take the number $ s = k^{-1} * (h + r * privKey) \mod n $, calculated during the signing process, we can calculate $ s1 = s^{-1} \mod n$ like this:  

$$ s1 = s^{-1} \mod n $$
$$ \equiv $$
$$ s1 = (k^{-1} * (h + r * privKey))^{-1} \mod n $$
$$ \equiv $$
$$ s1 = k * (h + r * privKey)^{-1} \mod n $$  

Now, replace **s1** in the point **R'**:  

$$ R' = (h + r * privKey) * s1 * G $$
$$ \equiv $$
$$ R' =  (h + r * privKey) * k * (h + r * privKey)^{-1} \mod n $$
$$ \equiv $$
$$ s1 = k * G $$  

In the end we get **k * G** which is the ***r*** value used to **Calculate the signature proof** in step 4 of ECDSA Sign shown above. 

## Lets get practical! 

In [1]:
import hashlib

In [ ]:
# Create our Elliptic Curve
E = EllipticCurve([-5, -1])
G = E.an_element()
n = 12345 # order of the curve (prime)

# Generate keys
privKey = 123
pubKey = privKey * G


def sign(msg, privKey): 
    # calculate msg hash
    hash = hashlib.sha256(msg.encode('utf8')).digest()
    # convert msg hash to integer
    int_hash = int.from_bytes(hash)

    k = randint(1, privKey-1)
    R = k*G
    Rx = R[0]
    proof = (k^-1 * (int_hash + Rx * privKey)) % n
    return (Rx, proof)

def verify(msg, signature, pubKey):
    # calculate msg hash
    hash = hashlib.sha256(msg.encode('utf8')).digest()
    # convert msg hash to integer
    int_hash = int.from_bytes(hash)
    s1 = power_mod(signature[1], -1, n)
    print(s1)
    Rlinha = (int_hash * s1)*G + (signature[0]*s1) * pubKey
    return Rlinha[0]

signature = sign("123", privKey)
R_verify = verify("123", signature, pubKey)

# Se a mensagem foi assinada com a privKey então o resultado tem de ser true
signature[0] == R_verify